In [1]:
import pandas as pd
import numpy as np
import sys

In [2]:
df = pd.read_csv("../summary_data/original/all_tools_TRB.csv")
df

,Sample,CDR3,nReads_TCR,nReads_CATT,nReads_MIXCR,nReads_IMREP,nReads_TRUST4,tissue,tissue_type,total_reads_TCR,...,total_reads_IMREP,total_reads_TRUST4,total_reads_CATT,frequency_TCR,frequency_MIXCR,frequency_IMREP,frequency_TRUST4,frequency_CATT,repertoire_type,class
0,SRR5233637,CASSPRVTSGTYEQYF,32.0,0.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,...,315.0,544.0,5616.0,0.000010,0.0,0.0,0.000000,0.0,high_SDI,T_cell_poor_high_SDI
1,SRR5233637,CASSYSDRGGQPQHF,13.0,0.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,...,315.0,544.0,5616.0,0.000004,0.0,0.0,0.000000,0.0,high_SDI,T_cell_poor_high_SDI
2,SRR5233637,CASKVALGGETQYF,25.0,0.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,...,315.0,544.0,5616.0,0.000008,0.0,0.0,0.000000,0.0,high_SDI,T_cell_poor_high_SDI
3,SRR5233637,CASRAPGTGTLGSPLHF,66.0,0.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,...,315.0,544.0,5616.0,0.000022,0.0,0.0,0.000000,0.0,high_SDI,T_cell_poor_high_SDI
4,SRR5233637,CASSSGQGGPSTEAFF,52.0,0.0,0.0,0.0,0.0,small_intestine,T_cell_poor,3047629.0,...,315.0,544.0,5616.0,0.000017,0.0,0.0,0.000000,0.0,high_SDI,T_cell_poor_high_SDI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378937,sample14,CASSESPAFGEKLFF,0.0,0.0,0.0,0.0,3.0,melanoma,T_cell_poor,749686.0,...,152.0,351.0,751.0,0.000000,0.0,0.0,0.008547,0.0,high_SDI,T_cell_poor_high_SDI
378938,sample14,CASSWTGSQETQYF,0.0,0.0,0.0,0.0,2.0,melanoma,T_cell_poor,749686.0,...,152.0,351.0,751.0,0.000000,0.0,0.0,0.005698,0.0,high_SDI,T_cell_poor_high_SDI
378939,sample14,CASRTGLAGGIGELFF,0.0,0.0,0.0,0.0,2.0,melanoma,T_cell_poor,749686.0,...,152.0,351.0,751.0,0.000000,0.0,0.0,0.005698,0.0,high_SDI,T_cell_poor_high_SDI
378940,sample14,CASSVEGYEQYF,0.0,0.0,0.0,0.0,2.0,melanoma,T_cell_poor,749686.0,...,152.0,351.0,751.0,0.000000,0.0,0.0,0.005698,0.0,high_SDI,T_cell_poor_high_SDI


In [3]:
RNA_seq_reads = pd.read_csv("../summary_data/original/RNA_seq_reads.csv")
RNA_seq_reads

,Sample,total_reads_RNA_seq
0,sample01,104984482
1,sample02,73892845
2,sample03,71654976
3,sample04,43179989
4,sample05,40524817
5,sample06,82476159
6,sample07,72397468
7,sample08,85492431
8,sample09,68584414
9,sample10,63320771


Total number of reads in each sample

In [4]:
total_reads = df[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_MIXCR','total_reads_IMREP','total_reads_TRUST4','total_reads_CATT']]
total_reads = total_reads.drop_duplicates(keep='first')
total_reads

# Add the column of RNA-Seq reads
total_reads = pd.merge(total_reads, RNA_seq_reads, how='left', on=['Sample'])
total_reads

# Calculate number of TCR derived reads per one million RNA-Seq reads 
# MIXCR
MIXCR = total_reads[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_MIXCR','total_reads_RNA_seq']]
MIXCR.rename(columns={'total_reads_MIXCR':'total_reads_tool'}, inplace=True)
MIXCR.loc[:,'tool'] = 'MIXCR'
MIXCR.loc[:,'TCR_derived_by_RNA_seq_tool'] = MIXCR['total_reads_tool']/MIXCR['total_reads_RNA_seq'] *1000000

# IMREP
IMREP = total_reads[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_IMREP','total_reads_RNA_seq']]
IMREP.rename(columns={'total_reads_IMREP':'total_reads_tool'}, inplace=True)
IMREP.loc[:,'tool'] = 'IMREP'
IMREP.loc[:,'TCR_derived_by_RNA_seq_tool'] = IMREP['total_reads_tool']/IMREP['total_reads_RNA_seq']*1000000

# TRUST4
TRUST4 = total_reads[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_TRUST4','total_reads_RNA_seq']]
TRUST4.rename(columns={'total_reads_TRUST4':'total_reads_tool'}, inplace=True)
TRUST4.loc[:,'tool'] = 'TRUST4'
TRUST4.loc[:,'TCR_derived_by_RNA_seq_tool'] = TRUST4['total_reads_tool']/TRUST4['total_reads_RNA_seq']*1000000

# CATT
CATT = total_reads[['Sample','tissue','tissue_type','repertoire_type','class','total_reads_TCR','total_reads_CATT','total_reads_RNA_seq']]
CATT.rename(columns={'total_reads_CATT':'total_reads_tool'}, inplace=True)
CATT.loc[:,'tool'] = 'CATT'
CATT.loc[:,'TCR_derived_by_RNA_seq_tool'] = CATT['total_reads_tool']/CATT['total_reads_RNA_seq']*1000000

reads_count = pd.concat([MIXCR,IMREP,TRUST4,CATT])
reads_count.to_csv('../summary_data/original/all_tools_TRB_reads.csv', index=False)

reads_count

/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1667: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value


,Sample,tissue,tissue_type,repertoire_type,class,total_reads_TCR,total_reads_tool,total_reads_RNA_seq,tool,TCR_derived_by_RNA_seq_tool
0,SRR5233637,small_intestine,T_cell_poor,high_SDI,T_cell_poor_high_SDI,3047629.0,84.0,72720367,MIXCR,1.155110
1,SRR5233639,lymph_node,T_cell_rich,high_SDI,T_cell_rich_high_SDI,3256697.0,1346.0,65998918,MIXCR,20.394274
2,TCGA-CZ-4862,kidney,T_cell_poor,high_SDI,T_cell_poor_high_SDI,16784.0,0.0,58529923,MIXCR,0.000000
3,TCGA-CZ-5463,kidney,T_cell_poor,high_SDI,T_cell_poor_high_SDI,806.0,0.0,122632451,MIXCR,0.000000
4,TCGA-CZ-5985,kidney,T_cell_poor,high_SDI,T_cell_poor_high_SDI,12998.0,0.0,83862519,MIXCR,0.000000
...,...,...,...,...,...,...,...,...,...,...
14,sample10,melanoma,T_cell_poor,high_SDI,T_cell_poor_high_SDI,1769522.0,1972.0,63320771,CATT,31.143019
15,sample11,melanoma,T_cell_poor,high_SDI,T_cell_poor_high_SDI,1006220.0,1739.0,55727841,CATT,31.205228
16,sample12,melanoma,T_cell_poor,high_SDI,T_cell_poor_high_SDI,292828.0,4985.0,107919183,CATT,46.191973
17,sample13,melanoma,T_cell_poor,low_SDI,T_cell_poor_low_SDI,12954.0,5052.0,80622502,CATT,62.662407


Average number of reads in different tissue types

In [5]:
mean_tissue_reads = reads_count.groupby("tissue")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
display(mean_tissue_reads)
tools = ['MIXCR','IMREP','TRUST4','CATT']              
for tool in tools: 
    print(tool)
    df_tool = reads_count.loc[reads_count['tool'] == tool]
    mean_tissue_reads = df_tool.groupby("tissue")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
    display(mean_tissue_reads)

/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


total_reads_TCR                total_reads_tool                \
                           mean            std             mean           std   
tissue                                                                          
PBMC               1.215376e+05   99453.125953     69952.500000  81323.548335   
kidney             1.019600e+04    7120.394473       320.750000    363.031710   
lymph_node         3.256697e+06       0.000000      5604.000000   3183.077442   
melanoma           9.512597e+05  520971.430765      1030.805556   1473.123384   
small_intestine    3.047629e+06       0.000000      1639.750000   2657.477046   

                total_reads_RNA_seq               TCR_derived_by_RNA_seq_tool  \
                               mean           std                        mean   
tissue                                                                          
PBMC                   6.684742e+07  2.419314e+07                  791.156788   
kidney                 8.834163e+07  2.753286e+07                    4.042224   
lymph_node             6.599892e+07  0.000000e+00                   84.910483   
melanoma               7.471916e+07  1.585619e+07                   13.284443   
small_intestine        7.272037e+07  0.000000e+00                   22.548704   

                             
                        std  
tissue                       
PBMC             791.106343  
kidney             4.851882  
lymph_node        48.229237  
melanoma          17.649304  
small_intestine   36.543779

MIXCR


/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


total_reads_TCR                total_reads_tool                \
                           mean            std             mean           std   
tissue                                                                          
PBMC               1.215376e+05  108376.531412     45224.200000  54187.320244   
kidney             1.019600e+04    8349.402613         0.000000      0.000000   
lymph_node         3.256697e+06            NaN      1346.000000           NaN   
melanoma           9.512597e+05  544844.963860       201.555556    339.992320   
small_intestine    3.047629e+06            NaN        84.000000           NaN   

                total_reads_RNA_seq               TCR_derived_by_RNA_seq_tool  \
                               mean           std                        mean   
tissue                                                                          
PBMC                   6.684742e+07  2.636386e+07                  510.665299   
kidney                 8.834163e+07  3.228514e+07                    0.000000   
lymph_node             6.599892e+07           NaN                   20.394274   
melanoma               7.471916e+07  1.658280e+07                    2.655950   
small_intestine        7.272037e+07           NaN                    1.155110   

                             
                        std  
tissue                       
PBMC             531.447948  
kidney             0.000000  
lymph_node              NaN  
melanoma           4.223584  
small_intestine         NaN

...

/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


total_reads_TCR                total_reads_tool  \
                           mean            std             mean   
tissue                                                            
PBMC               1.215376e+05  108376.531412     91349.200000   
kidney             1.019600e+04    8349.402613       452.666667   
lymph_node         3.256697e+06            NaN      9054.000000   
melanoma           9.512597e+05  544844.963860       665.333333   
small_intestine    3.047629e+06            NaN       544.000000   

                               total_reads_RNA_seq                \
                           std                mean           std   
tissue                                                             
PBMC             107795.349996        6.684742e+07  2.636386e+07   
kidney              226.780364        8.834163e+07  3.228514e+07   
lymph_node                 NaN        6.599892e+07           NaN   
melanoma            888.407846        7.471916e+07  1.658280e+07   
small_intestine            NaN        7.272037e+07           NaN   

                TCR_derived_by_RNA_seq_tool               
                                       mean          std  
tissue                                                    
PBMC                            1036.619505  1050.450769  
kidney                             6.159241     5.174758  
lymph_node                       137.184067          NaN  
melanoma                           8.780548    11.009198  
small_intestine                    7.480710          NaN

CATT


/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


total_reads_TCR                total_reads_tool                \
                           mean            std             mean           std   
tissue                                                                          
PBMC               1.215376e+05  108376.531412     74583.800000  86529.239077   
kidney             1.019600e+04    8349.402613       800.666667    141.796803   
lymph_node         3.256697e+06            NaN      5873.000000           NaN   
melanoma           9.512597e+05  544844.963860      3041.444444   1516.156911   
small_intestine    3.047629e+06            NaN      5616.000000           NaN   

                total_reads_RNA_seq               TCR_derived_by_RNA_seq_tool  \
                               mean           std                        mean   
tissue                                                                          
PBMC                   6.684742e+07  2.636386e+07                  855.609168   
kidney                 8.834163e+07  3.228514e+07                    9.589991   
lymph_node             6.599892e+07           NaN                   88.986307   
melanoma               7.471916e+07  1.658280e+07                   38.847473   
small_intestine        7.272037e+07           NaN                   77.227333   

                             
                        std  
tissue                       
PBMC             827.461979  
kidney             2.338000  
lymph_node              NaN  
melanoma          14.337259  
small_intestine         NaN

Average number of reads in different repertoire types

In [6]:
mean_repertoire_reads = reads_count.groupby("class")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
display(mean_repertoire_reads)
tools = ['MIXCR','IMREP','TRUST4','CATT']              
for tool in tools: 
    print(tool)
    df_tool = reads_count.loc[reads_count['tool'] == tool]
    mean_repertoire_reads = df_tool.groupby("class")["total_reads_TCR","total_reads_tool","total_reads_RNA_seq","TCR_derived_by_RNA_seq_tool"].agg(["mean", "std"]) 
    display(mean_repertoire_reads)

/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


total_reads_TCR               total_reads_tool  \
                                mean           std             mean   
class                                                                 
T_cell_poor_high_SDI    9.688833e+05  8.526509e+05       785.312500   
T_cell_poor_low_SDI     1.295400e+04  0.000000e+00      2455.500000   
T_cell_rich_high_SDI    1.126698e+06  1.573538e+06      2671.666667   
T_cell_rich_low_SDI     1.614307e+05  1.067436e+05    115783.833333   

                                   total_reads_RNA_seq                \
                               std                mean           std   
class                                                                  
T_cell_poor_high_SDI   1334.452572        7.746627e+07  2.004225e+07   
T_cell_poor_low_SDI    1950.656727        8.062250e+07  0.000000e+00   
T_cell_rich_high_SDI   2949.090933        4.990124e+07  1.194271e+07   
T_cell_rich_low_SDI   75454.896953        8.351077e+07  1.588807e+07   

                     TCR_derived_by_RNA_seq_tool              
                                            mean         std  
class                                                         
T_cell_poor_high_SDI                   10.314884   16.744026  
T_cell_poor_low_SDI                    30.456758   24.194942  
T_cell_rich_high_SDI                   47.178014   45.627414  
T_cell_rich_low_SDI                  1299.720126  612.346881

MIXCR


/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


total_reads_TCR               total_reads_tool  \
                                mean           std             mean   
class                                                                 
T_cell_poor_high_SDI    9.688833e+05  8.812393e+05        68.916667   
T_cell_poor_low_SDI     1.295400e+04           NaN      1071.000000   
T_cell_rich_high_SDI    1.126698e+06  1.845137e+06       519.666667   
T_cell_rich_low_SDI     1.614307e+05  1.251679e+05     75302.666667   

                                   total_reads_RNA_seq                \
                               std                mean           std   
class                                                                  
T_cell_poor_high_SDI     92.146581        7.746627e+07  2.071424e+07   
T_cell_poor_low_SDI            NaN        8.062250e+07           NaN   
T_cell_rich_high_SDI    717.476364        4.990124e+07  1.400407e+07   
T_cell_rich_low_SDI   49798.098451        8.351077e+07  1.863041e+07   

                     TCR_derived_by_RNA_seq_tool              
                                            mean         std  
class                                                         
T_cell_poor_high_SDI                    0.981210    1.331736  
T_cell_poor_low_SDI                    13.284133         NaN  
T_cell_rich_high_SDI                    8.470190   10.390502  
T_cell_rich_low_SDI                   849.436733  366.742010

...

/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


total_reads_TCR               total_reads_tool  \
                                mean           std             mean   
class                                                                 
T_cell_poor_high_SDI    9.688833e+05  8.812393e+05       419.583333   
T_cell_poor_low_SDI     1.295400e+04           NaN      2855.000000   
T_cell_rich_high_SDI    1.126698e+06  1.845137e+06      3748.666667   
T_cell_rich_low_SDI     1.614307e+05  1.251679e+05    151518.000000   

                                   total_reads_RNA_seq                \
                               std                mean           std   
class                                                                  
T_cell_poor_high_SDI    308.670598        7.746627e+07  2.071424e+07   
T_cell_poor_low_SDI            NaN        8.062250e+07           NaN   
T_cell_rich_high_SDI   4683.804152        4.990124e+07  1.400407e+07   
T_cell_rich_low_SDI   98299.440543        8.351077e+07  1.863041e+07   

                     TCR_derived_by_RNA_seq_tool              
                                            mean         std  
class                                                         
T_cell_poor_high_SDI                    5.797618    4.567406  
T_cell_poor_low_SDI                    35.411950         NaN  
T_cell_rich_high_SDI                   62.743517   67.780944  
T_cell_rich_low_SDI                  1710.683680  708.940058

CATT


/Users/keruipeng/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  import sys


total_reads_TCR               total_reads_tool  \
                                mean           std             mean   
class                                                                 
T_cell_poor_high_SDI    9.688833e+05  8.812393e+05      2528.250000   
T_cell_poor_low_SDI     1.295400e+04           NaN      5052.000000   
T_cell_rich_high_SDI    1.126698e+06  1.845137e+06      3882.333333   
T_cell_rich_low_SDI     1.614307e+05  1.251679e+05    122381.666667   

                                   total_reads_RNA_seq                \
                               std                mean           std   
class                                                                  
T_cell_poor_high_SDI   1729.921234        7.746627e+07  2.071424e+07   
T_cell_poor_low_SDI            NaN        8.062250e+07           NaN   
T_cell_rich_high_SDI   2079.575518        4.990124e+07  1.400407e+07   
T_cell_rich_low_SDI   80036.745588        8.351077e+07  1.863041e+07   

                     TCR_derived_by_RNA_seq_tool              
                                            mean         std  
class                                                         
T_cell_poor_high_SDI                   32.746846   20.632367  
T_cell_poor_low_SDI                    62.662407         NaN  
T_cell_rich_high_SDI                   75.107196   28.304669  
T_cell_rich_low_SDI                  1380.570187  579.042056